<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w2d3_t1_initial/tutorials/W2D3_ModernRecurrentNeuralNetworks/W2D3_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy: Week 2, Day 3, Tutorial 1
# Modern RNNs

__Content creators:__ James Evans, Bhargav Srinivasa Desikan, Anis Zahedifard

__Content reviewers:__ XXXXX, XXXXXX

__Content editors:__ XXXXX

__Production editors:__ Roberto Guidotti, Spiros Chavlis, XXXXXX




In [1]:
# This is a place holder notebook. DO NOT USE.